# GPU内存

In [1]:
import torch

if torch.cuda.is_available():
    torch.cuda.empty_cache()

if torch.cuda.is_available():
    print(f"当前已分配显存: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"当前缓存中显存: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
else:
    print("No GPU!")

当前已分配显存: 0.00 GB
当前缓存中显存: 0.00 GB


# 1 准备

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# 定义 max_memory 参数
# 键是设备编号 (0 代表第一张 GPU)，值是内存大小
# 你可以使用 "GiB", "MiB", "GB", "MB" 等单位
max_memory_map = {0: "15GiB"} 


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings

# --- 1. 保持您原有的模型和分词器加载方式 ---
# 这部分完全不变，因为它正确地使用了 transformers 的功能来加载模型到 GPU
print("正在加载模型和分词器...")
HF_CACHE_DIR = "/root/nas-private/bigdata_final_project/models/huggingface_cache"
LLM_RELATIVE_PATH = "/root/nas-private/bigdata_final_project/models/Qwen2.5-7B-Instruct"

LOCAL_LLM_PATH = os.path.join(HF_CACHE_DIR, LLM_RELATIVE_PATH)
model_name = LOCAL_LLM_PATH

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 确保在加载模型时处理可能的信任远程代码问题
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    max_memory=max_memory_map,
    #torch_dtype="auto", # 自动选择最佳精度 (如 bfloat16)
    dtype= "auto",
    device_map="auto",  # 智能地将模型分布到所有可用 GPU
    trust_remote_code=True # Qwen 系列模型通常需要这个
)
print("模型和分词器加载完成。")


正在加载模型和分词器...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

模型和分词器加载完成。


In [4]:

# --- 2. 将加载好的对象包装进 HuggingFaceLLM ---
# 这是关键的改写步骤
print("正在将模型包装到 LlamaIndex 的 HuggingFaceLLM 中...")

# 创建 HuggingFaceLLM 实例
llm = HuggingFaceLLM(
    # 传入已经加载好的模型和分词器对象
    model=model,
    tokenizer=tokenizer,
    
    # 提供模型名称，用于元数据记录
    model_name=model_name,
    
    # 关键参数：告知 LlamaIndex 模型的上下文窗口大小
    # Qwen3-8B 的上下文窗口非常大，这里设置为一个合理的值，例如 32768
    context_window= 8192, #32768,
    
    # 控制生成文本的最大长度
    max_new_tokens=1024,   #512,
    # 传递给 transformers.generate() 的额外参数
    generate_kwargs={"do_sample": True, "temperature": 0.7, "top_k": 50},
    
    # 告知包装器模型已经通过 device_map 分布在设备上
    device_map="auto"
)

print("HuggingFaceLLM 包装完成。")

正在将模型包装到 LlamaIndex 的 HuggingFaceLLM 中...
HuggingFaceLLM 包装完成。


In [5]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# 模型的相对路径（不带开头的 ./，保持路径结构清晰）
EMBED_RELATIVE_PATH = "/root/nas-private/bigdata_final_project/models/bge-small-zh-v1.5"
LOCAL_EMBED_PATH =os.path.join(HF_CACHE_DIR,EMBED_RELATIVE_PATH)
# 初始化本地 Embedding Model
embed_model = HuggingFaceEmbedding(
    model_name=LOCAL_EMBED_PATH, # 注意这里改为本地路径
)

# 设置嵌入模型
Settings.embed_model = embed_model

# 设置LLM模型
Settings.llm =llm

In [6]:
QA_PROMPT_TEMPLATE_STR = (
    "You are a helpful assistant. Please answer the user's question based on the provided context.\n\n"
    "Context: {context_str}\n"
    "---------------------\n"
    "Question: {query_str}\n"
    "Answer: "
)

In [7]:
from llama_index.core import VectorStoreIndex, PromptTemplate
qa_prompt_template = PromptTemplate(QA_PROMPT_TEMPLATE_STR)

# 2  chat

## 2.1 complete 模式



In [8]:
# 这种方法只适用于单次、无历史的提示
prompt = "写一个关于中国长城的小故事，不超过100字。"

# 使用 .complete() 方法（如果您的 HuggingFaceLLM 实例支持）
# 注意：通常 chat() 是推荐用于与 Chat 模型交互的方式，但 complete() 也可以用于简单的提示
try:
    response = Settings.llm.complete(prompt)
    print("--- LLM Response (Complete) ---")
    print(response.text)
    print("-------------------------------\n")
except AttributeError:
    print("Settings.llm 实例不支持 .complete() 方法，请使用包装后的 .chat()。")


--- LLM Response (Complete) ---
 从前，皇帝为抵御北方的入侵者，命令无数工匠和士兵修建长城。一位年轻的石匠在修筑长城时，与邻近村庄的姑娘相爱了。但战争不断，姑娘的父亲要求石匠离开。石匠将自己雕刻的一块心形石块留在城墙上，以示对姑娘的思念。多年后，战争结束，石匠回到家乡，找到了那块心形石块，重新与心爱的人团聚。这象征着长城不仅是防御工事，也是连接人心的桥梁。
-------------------------------



## 2.2 chat 模式

### 2.2.1 一次

In [9]:
# 导入 ChatMessage，用于构造对话历史
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings # 确保 Settings 在当前作用域可用

# 假设您想问一个关于中国的简单问题
prompt_content = "写一个关于中国长城的小故事，不超过100字。"

print(f"User Prompt: {prompt_content}\n")

# **关键修改：将字符串包装成 ChatMessage 列表**
messages = [
    ChatMessage(role=MessageRole.USER, content=prompt_content)
]

# 直接调用 llm 实例的 chat 方法，传入 messages 列表
response = Settings.llm.chat(messages)

print("--- LLM Response (Chat) ---")
print(response.message.content)
print("---------------------------\n")

User Prompt: 写一个关于中国长城的小故事，不超过100字。

--- LLM Response (Chat) ---
在遥远的明朝，小村落里流传着长城能保佑平安的传说。一天，村中少年李明偶得一块刻有神秘符号的石片，得知只有将它投入长城的最北端，才能真正守护家园。于是，他踏上寻石之旅，历经千辛万苦终于抵达。当石片投入城砖间那一刻，夕阳洒落，长城仿佛被赋予了新的生命力，更显巍峨壮丽。
---------------------------



### 2.2.2 多次

In [10]:
from llama_index.core.chat_engine import SimpleChatEngine

# 使用 Settings 中配置的 llm 创建 ChatEngine
chat_engine = SimpleChatEngine.from_defaults(
    llm=Settings.llm,
    # System Prompt 可选，用于设定模型角色
    system_prompt="你是一个乐于助人、充满好奇心的 AI 助手，你的回答总是简洁明了。",
)

print("--- Chat Engine Conversation ---")

# 第一次提问
response_1 = chat_engine.chat("给我推荐一个适合在秋天阅读的中文小说名字。")
print(f"Assistant: {response_1.response}\n")

# 第二次提问（ChatEngine 会自动维护历史）
response_2 = chat_engine.chat("这个作者还有其他的作品吗？")
print(f"Assistant: {response_2.response}\n")

print("--------------------------------\n")

--- Chat Engine Conversation ---
Assistant: 《白鹿原》是一部非常适合秋天阅读的小说，它描绘了中国农村的生活变迁，文字细腻而富有诗意，非常适合在这个收获的季节细细品味。

Assistant: 是的，陈忠实除了《白鹿原》之外，还写有《信任》《初夏》等短篇小说集，以及散文集《陈忠实自选集》等。如果你喜欢《白鹿原》，可以尝试阅读他的其他作品。

--------------------------------



## 3 工具调用

In [11]:
llm.metadata

LLMMetadata(context_window=8192, num_output=1024, is_chat_model=False, is_function_calling_model=False, model_name='/root/nas-private/bigdata_final_project/models/Qwen2.5-7B-Instruct', system_role=<MessageRole.SYSTEM: 'system'>)

In [12]:
import asyncio
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import FunctionTool
from llama_index.core import Settings # 您的 LLM 在 Settings.llm 中
# ... (您的 LLM 和 Settings.llm 配置已完成) ...

# ----------------- 定义工具 (假设已成功) -----------------
def get_current_weather(city: str) -> str:
    """获取指定城市（例如：'北京'，'上海'）的当前天气信息。"""
    if "北京" in city:
        return "北京今天的气温是 28°C，多云转晴，空气质量良好。"
    elif "上海" in city:
        return "上海今天的气温是 -18°C，下雨，空气质量一般。"
    return f"抱歉，没有 {city} 的天气数据。"

def multiply(a:int, b:int ) -> int :
    """将两个数相乘(乘法运算)"""
    return a * b + 10000

weather_tool = FunctionTool.from_defaults(fn=get_current_weather)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
tools = [weather_tool, multiply_tool]


agent = ReActAgent(tools=tools, llm=Settings.llm, verbose=True)


In [13]:

# ----------------- 定义异步执行函数 -----------------
async def run_agent_queries(agent_instance: ReActAgent):
    print("--- 异步调用 1: 工具函数 (乘法) ---")
    agent_query_calc = "调用乘法工具，计算一下 50 乘以 91 的结果是多少？"
    # 使用 .arun() 方法进行异步执行
    response_calc = await agent_instance.run(agent_query_calc)
    print(f"最终回答: {response_calc.response}\n")

    print("--- 异步调用 2: 工具函数 (天气) ---")
    agent_query_weather = "告诉我上海今天的天气怎么样？"
    response_weather = await agent_instance.run(agent_query_weather)
    print(f"最终回答: {response_weather.response}\n")
    
await run_agent_queries(agent)

--- 异步调用 1: 工具函数 (乘法) ---
最终回答: assistant: 50 乘以 91 的结果是 14550。

--- 异步调用 2: 工具函数 (天气) ---
最终回答: assistant: 上海今天的天气情况是气温 -18°C，并且有雨，空气质量一般。请注意保暖并关注实时天气变化。



# Rag1

In [16]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader("/root/nas-private/bigdata_final_project/demos/txt1").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt_template    
)
response = query_engine.query(  "邓国标工作单位是什么，复旦教育经历的是什么")
print(response)

依据提供的信息，邓国标的信息并未出现在文本中，因此无法回答其工作单位和复旦教育经历。文本中提到的校友是常怀春和施小琳，其中常怀春的工作单位是“山东华鲁恒升化工股份有限公司”，复旦教育经历为“复旦EMBA;班级：2008秋1班”。而施小琳的工作单位则为“四川省省委/副书记;四川省政府/党组书记;四川省人民政府/省长”。 对于邓国标的相关信息，需要提供相关文件或数据以进行准确回答。 根据提供的信息，答案如下：

- 常怀春的工作单位是山东华鲁恒升化工股份有限公司。
- 常怀春的复旦教育经历是复旦EMBA，班级为2008秋1班。 没有提到邓国标的相关信息。 施小琳的工作单位是四川省省委/副书记; 四川省政府/党组书记; 四川省人民政府/省长。 施小琳的复旦教育经历为复旦管院政府培训班。 对于邓国标的信息，无法从给定的内容中获取。 需要更多关于邓国标的资料来进行回答。 

请注意，上述答案基于提供的信息。如果需要更详细的信息，建议查找相关的详细资料。如果有更多的文档或者数据，请提供，以便给出更准确的答案。


In [17]:
nodes=index.as_query_engine(llm=Settings.llm,text_qa_template=qa_prompt_template).query("用中文回答，施小琳教育经历")

In [18]:
print(nodes)

以下是施小琳的教育经历：

- 复旦管院政府培训班（1998年）
- 上海市高级管理干部培训班（百人工程）第五期
- 高端培训

这些信息显示了她在政府管理和高级管理方面的培训背景。


In [19]:
len(nodes.source_nodes)

2

In [20]:
nodes.source_nodes[0].text

'以下内容与施小琳有关\n姓名：施小琳\n性别：女\n婚姻状况\t其他\n居住地\t上海市\n更新时间\t2023/03/01\n施小琳标签\t两会代表, 院长级, 二十大人大代表, 第十四届全国人大代表（2023）\n施小琳项目\t政府培训班\n\n施小琳教育经历：复旦管院政府培训班/1998上海市高级管理干部培训班(百人工程)第五期/高端培训\t\n\n施小琳当前工作(工作单位/部门/职位)：四川省省委/副书记;四川省政府/党组书记;四川省人民政府/省长\n\n行业\t政府机构/非营利机构;政府机构/非营利机构;政府机构/非营利机构\n\n二类行业\t政府机构/非营利机构;政府机构/非营利机构;政府机构/非营利机构\n\n---\n\n施小琳\n工作履历(工作单位/部门/职位)\t上海市普陀区人民政府/区委书记、区委委员;上海市民政局/局长;南汇区人民政府/副区长;虹口区人民政府/副区长;上海市委常委/统战部部长;江西省委/常委;江西省宣传部/部长;四川省成都市/成都市委书记、兼任成都警备区党委第一书记;四川省人民政府/副省长、代理省长\n\n政府类职级\t省部级副职;省部级副职;省部级副职;省部级副职;省部级正职'

In [21]:
from llama_index.core import PromptTemplate

text_qa_template_str = (
    "Context information is"
    " below.\n---------------------\n{context_str}\n---------------------\n only Using"
    " the context information, not using your own knowledge, answer"
    " the question: {query_str}\nIf the context isn't helpful, you will not "
    " answer the question on your own.\n 所有回答以中文显示 \n"
)
text_qa_template = PromptTemplate(text_qa_template_str)

refine_template_str = (
    "The original question is as follows: {query_str}\nWe have provided an"
    " existing answer: {existing_answer}\nWe have the opportunity to refine"
    " the existing answer (only if needed) with some more context"
    " below.\n------------\n{context_msg}\n------------\nUsing the new"
    " context , update or repeat the existing answer.\n 所有回答以中文显示 \n"
)
refine_template = PromptTemplate(refine_template_str)

In [22]:
res=index.as_query_engine(
        text_qa_template=text_qa_template,
        refine_template=refine_template,
        llm=llm,
    ).query("按时间顺序，列出施小琳的所有职务变更，如果有时间也列出来。")
print(res.response)

根据提供的信息，按时间顺序列出施小琳的所有职务变更如下：

1. 1990年7月参加工作
2. 上海市民政局局长、党组书记
3. 上海市普陀区委书记
4. 上海市委常委、统战部部长
5. 江西省委常委、宣传部部长
6. 四川省委常委、成都市委书记
7. 2023年7月，任四川省委副书记
8. 2024年7月4日，任四川省副省长、代理省长
9. 2024年7月31日，任四川省人民政府省长
10. 2024年6月28日，任四川省政府党组书记

注意：具体参加工作的年份未明确给出，因此将其列为最早的时间点。其余职务变更的时间均依据提供的信息。


# Rag2

In [23]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.agent import ReActAgent

In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
documents = SimpleDirectoryReader("/root/nas-private/bigdata_final_project/demos/txt2").load_data()
index = VectorStoreIndex.from_documents(
    documents,
)
query_engine = index.as_query_engine(
  text_qa_template=qa_prompt_template    
)


In [25]:
# 2. 封装为 QueryEngineTool
rag_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="financial_principle_knowledge",
        description=(
            "当用户询问**基础投资原则、风险等级、市场分类**等知识时，使用此工具。 "
            "它提供来自内部金融文档的背景信息和规则。"
        ),
    ),
)


# --- B. ML/量化分析工具：模拟模型预测 ---

def conduct_ml_analysis(asset_type: str, risk_level: str) -> str:
    """
    调用金融机器学习模型（例如，风险预测、收益率模拟），分析特定资产在特定风险下的潜在表现。
    输入 asset_type 必须是 '股票', '债券', '基金' 中的一种。
    输入 risk_level 必须是 '保守', '稳健', '激进' 中的一种。
    返回分析报告的摘要。
    """
    
    # 模拟复杂的模型运行结果
    if risk_level == '激进' and asset_type == '股票':
        result = "机器学习模型预测：高成长科技股在未来12个月内有 75% 的概率获得 20% 以上的年化收益，但其最大回撤风险为 35%。建议严格控制仓位。"
    elif risk_level == '保守' and asset_type == '债券':
        result = "模型分析：高等级企业债券在当前市场环境下流动性较好，预计年化收益在 4%-6% 之间，风险极低，适合保值。"
    elif risk_level == '稳健' and asset_type == '基金':
        result = "模型预测：平衡型混合基金在当前波动市场中表现稳健，模型给予‘增持’评级，建议配置 40% 的资产。"
    else:
        result = f"模型对 {risk_level} 投资者配置 {asset_type} 的分析结果暂无高置信度报告，建议结合RAG知识库中的原则进行配置。"
        
    return result

# 封装为 FunctionTool
ml_analysis_tool = FunctionTool.from_defaults(fn=conduct_ml_analysis)


# --- C. 构建 ReAct Agent ---
agent = ReActAgent(
    tools=[rag_tool, ml_analysis_tool],
    llm=Settings.llm,
    verbose=True, # 开启 verbose，观察 Qwen 的思考过程
    # 设置一个合适的系统提示，引导 Qwen 扮演金融顾问的角色
    system_prompt="你是一位专业的金融投资顾问。你的任务是根据用户的风险偏好和资产类型，综合利用内部知识库（financial_principle_knowledge）和机器学习模型分析（conduct_ml_analysis）来给出个性化的、负责任的投资建议。",
)
print("金融 Agent 创建完成，拥有 RAG 和 ML 分析能力。")

金融 Agent 创建完成，拥有 RAG 和 ML 分析能力。


In [26]:
async def run_finance_queries(agent_instance: ReActAgent,query:str):
    #print(f"\n[ Agent Query : {query} ]")
    response = await agent_instance.run(query)
    #print("\n" + "="*50)
    #print(f"Agent 最终回答 A:\n{response.response}") 
    #print("="*50 + "\n")
    return response.response
async def ask(q):
    # 必须使用 await 调用异步函数
    r = await run_finance_queries(agent, q)
    return r.blocks[0].text
    
q_string="我是一个激进型投资者，请问基于内部文档的原则和机器学习模型分析，我应该如何配置高成长性股票？"    
res= await ask(q_string)

In [27]:
res

'激进型投资者可以根据以下建议来配置高成长性股票：\n\n1. 将超过50%的资金配置到高成长性股票上，特别是高成长科技股。\n2. 重点关注具有较强增长潜力的股票，尤其是那些在未来12个月内有75%的概率获得20%以上年化收益的股票。\n3. 注意控制仓位，因为这些股票的最大回撤风险为35%，需要严格控制风险。\n\n总之，激进型投资者应该积极寻找高增长潜力的股票进行投资，但在投资过程中需要密切关注市场动态，及时调整投资组合，并严格控制仓位以降低风险。希望以上建议对您有所帮助。如果有任何疑问或需要进一步的信息，请随时联系我。祝您投资顺利！'

In [28]:
q_string="明天的天气？"    
res= await ask(q_string)
print(res)

对不起，我目前没有查询天气的功能。你可以尝试使用其他天气查询服务来获取明天的天气预报。
